In [21]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import clear_output 

In [29]:
model = load_model("mp_last_large_map")

In [23]:
def preprocess(landmark):
    processed_landmarks = []
    relative_landmarks= []
    for i in landmark[0].landmark:
        processed_landmarks.append([i.x, i.y])

    for i in processed_landmarks:
        relative_landmarks.append([
            i[0]-processed_landmarks[0][0],
            i[1]-processed_landmarks[0][1],
        ])
    
    flatten = []
    for i in relative_landmarks:
        flatten.append(i[0])
        flatten.append(i[1])
    
    maxVal = max(list(map(abs, flatten)))
    def norm(n): return n/maxVal
    normalised = list(map(norm, flatten))

    return normalised

In [24]:
encoding_maps = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
       'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 
       'U', 'V', 'W', 'X', 'Y', 'Z']

In [32]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
    # while True:
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                        )

            res = preprocess(results.multi_hand_landmarks)
            prob = model.predict([res])
            pred = np.argmax(prob)
            maxi = prob[0][pred]
            tresh = .98
            sign = encoding_maps[pred]
            clear_output()
            print("res:", sign, "max - prob:", maxi)
            if maxi > tresh:
                image = cv2.putText(image, sign, org=(50, 50), 
                                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                    fontScale=1, color=(255,255,255), 
                                    thickness=2)

        cv2.imshow('Hand Tracking', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


-1